In [ ]:
# ! pip install pyresample
# ! pip install xarray[complete]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import xarray
import cartopy.crs as ccrs
import cartopy.feature as cf
import subprocess

import datetime as dt

import pyresample.geometry as pgeom
import pyresample.utils as putils
import pyresample.kd_tree as pkdt
from collections import OrderedDict

In [ ]:
#date = dt.datetime.now().strftime('%Y%m%d00') # string for today 00utc
date='2025041400'
date

# Let's download some T2M grib data LT=12hrs, init: today 0UTC

In [ ]:
t2mfile = f"icon_global_icosahedral_single-level_{date}_012_T_2M.grib2.bz2"

In [ ]:
#! wget "http://opendata.dwd.de/weather/nwp/icon/grib/00/t_2m/{t2mfile}"

In [ ]:
#!bzip2 -d "{t2mfile}"

In [ ]:
#!wget http://icon-downloads.mpimet.mpg.de/grids/public/edzw/icon_grid_0026_R03B07_G.nc

# cfgrib backend of xarray: easy to use, but can be slooooooow

In [ ]:
ds = xarray.open_dataarray(f'./icon_global_icosahedral_single-level_{date}_012_T_2M.grib2',engine='cfgrib', decode_timedelta=False)
ds = ds.rename(values='cell')

In [ ]:
grid = xarray.open_dataset('./icon_grid_0026_R03B07_G.nc')

In [ ]:
full = xarray.Dataset(dict(T2M=ds), coords=dict(latitude=grid.clat, longitude=grid.clon))
full['longitude']  = np.rad2deg(full["clon"])
full['latitude']  = np.rad2deg(full["clat"])

In [ ]:
full

## simplest way to plot triangular data without fancy library

In [ ]:
# example from the ICON trainings course
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(1, 1, subplot_kw={"projection":crs})

def axis_settings(ax, title):
    gl = ax.gridlines(crs=crs, draw_labels=True,
                      linewidth=.6, color='gray')
    ax.add_feature(cf.COASTLINE.with_scale("50m"), lw=0.5)
    ax.add_feature(cf.BORDERS.with_scale("50m"), lw=0.3)
    ax.set_title(title)
    
axis_settings(ax, "T2M")
filled_c0 = ax.tricontourf(full.longitude, full.latitude, full["T2M"][:], transform=crs)
fig.colorbar(filled_c0, orientation='horizontal', ax=ax);

## more sophisticated: plot triangles

In [ ]:
def create_tri(grid):
    """
    grid: xarray.Dataset from gridfile
    """
    # ! 1 based indexing in grid file
    return mtri.Triangulation(180./np.pi*grid['vlon'],180./np.pi*grid['vlat'],(grid['vertex_of_cell']-1).T)

def plot_tri(da,grid, ax=None, **kwargs):
    # expect da=xarray.DataArray with grid file coords in da.coords
    # kwargs go to tripcolor

    triangulation = create_tri(grid)
    triangulation, values = fix_dateline_triangles(triangulation, da.values, False)
    subplotkw = dict(projection=ccrs.PlateCarree())
    if ax is None:
        fig,ax = plt.subplots(1, subplot_kw=subplotkw)
    cax = ax.tripcolor(triangulation, values, **kwargs)
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    return ax,triangulation,cax
    
def fix_dateline_triangles(triangulation, values, mask_only=False): # gernot
    """Fix triangles crossing the date line.

    Triangles crossing the horizontal map boundary are plotted across
    the whole two-dimensional plane of the PlateCarree projection
    (eg. from -180 degrees to 180 degrees), while they should "wrap
    around the back". For the respective triangles on either side of
    the plot, the vertices beyond the date line - on thus on the
    opposite side of the plot - are re-set to a value on the same side
    and the triangle is duplicated on the other side.

    To visualize this effect, use mask_only=True. In this case, the
    triangles are not duplicated and the respective triangles are only
    masked and will not be plotted.
    
    (ideas taken from the ICON Model Tutorial 2019, section 9.3.3)
        
    Parameters
    ----------
    triangulation : Triangulation
        the triangulation to be fixed
    values : ndarray
        the values corresponding to the triangulation
    mask_only : bool, optional
        whether to mask the triangles without changing the vertices
            
    Returns     
    -------                                       
    triangulation_fixed : Triangulation 
        the triangulation with modified triangles and vertices
    values_fixed : ndarray
        the values with duplicated values for duplicated triangles appended

    """
    to_fix = np.argwhere(triangulation.x[triangulation.triangles].max(axis=1) 
                                               - triangulation.x[triangulation.triangles].min(axis=1) > 200)[:, 0]

    # create a new Triangulation object to avoid overwriting the original data
    triangulation_fixed = mtri.Triangulation(triangulation.x, triangulation.y, triangulation.triangles)

    if mask_only:
        triangulation_fixed.mask = np.full(triangulation.triangles.shape[0], False)
        triangulation_fixed.mask[to_fix] = True
    else:
        values_fixed = values.copy()
        k = triangulation.x.shape[0]
        for i in to_fix:
            # append the mirrored triangle and its value to the existing triangles and values
            triangle = triangulation.triangles[i]
            triangulation_fixed.triangles = np.vstack([triangulation_fixed.triangles, triangle])
            values_fixed = np.append(values_fixed, values[i])

            # adjust the vertices of the appended triangle such that all lon values are > 0
            idx_vertex = np.argwhere(triangulation.x[triangle]<0)
            for j in idx_vertex:
                triangulation_fixed.x = np.append(triangulation_fixed.x,
                                                  triangulation.x[triangle[j]] + 360)
                triangulation_fixed.y = np.append(triangulation_fixed.y,
                                                  triangulation.y[triangle[j]])
                triangulation_fixed.triangles[-1, j] = k
                k = k+1

            # adjust the vertices of the original, copied triangle such that all lon values are < 0
            idx_vertex = np.argwhere(triangulation.x[triangle]>0)
            for j in idx_vertex:
                triangulation_fixed.x = np.append(triangulation_fixed.x,
                                                  triangulation.x[triangle[j]] - 360)
                triangulation_fixed.y = np.append(triangulation_fixed.y,
                                                  triangulation.y[triangle[j]])
                triangulation_fixed.triangles[i, j] = k
                k = k+1

    return triangulation_fixed, values_fixed

In [ ]:
ax, tri, cax = plot_tri(full['T2M'], grid,vmin=270,vmax=290);
ax.set_extent([5,10,55,60])
plt.colorbar(cax,label='T2M', shrink=0.6)
plt.tight_layout()

## remap to custom points: here just one meridian

In [ ]:
def product(*args):
    """Return a generator for the Cartesian product of args
    (https://confluence.ecmwf.int/display/ECC/grib_index, Python tab).

    Parameters
    ----------
    args : sequence of sequences

    Returns
    -------
    generator
        yielding tuples from the Cartesian product of args

    """

    result = [[]]
    for pool in args:
        result = [x + [y] for x in result for y in pool]
    for i in result:
        yield tuple(i)


method='gauss'; sigma=20; cutoff=None

def compute_xsection(da, lllat, lllon, method='gauss', sigma=20, cutoff=None, lev=None):
        """
        Interpolation to pairs of lllat,lllon values (uses pyresample!)
        
        da :            xarray.DataArray, to be interpolated. 
                         Required coords: longitude, latitude (deg), cells
        lllat,lllon :   xarray.DataArray, 1D target coordinates (deg)
        method :         str, options: 'gauss', 'nearest'
        sigma, cutoff : only for method='gauss'
        lev :           xarray.DataArray (1d), optional. 
                        expected to be called 'height', in target array, can be renamed otherwise
        """

        if lev is not None:
            target = xarray.zeros_like(lev+lllat) # hack to get z dimension!
        else:
            target = xarray.zeros_like(lllat)
        target.name = da.name


        source_geo = pgeom.SwathDefinition(
                        *putils.check_and_wrap(da.coords['longitude'].values,
                                               da.coords['latitude'].values
                                               )
                                           )

        target_geo = pgeom.SwathDefinition(
                        *putils.check_and_wrap(target.coords['longitude'].values,
                                               target.coords['latitude'].values
                                               )
                                           )

        if method == 'gauss':
            sigma = sigma * 1000
            if cutoff is None:
                cutoff = 5 * sigma
            resample_args = [cutoff, sigma]
            resample_func = getattr(pkdt, 'resample_gauss')

        elif method == 'nearest':
            if cutoff is None:
                cutoff = 50000
            resample_args = [cutoff]
            resample_func = getattr(pkdt, 'resample_nearest')

        # derive dimensions and their sizes of the result
        # 1) take da.sizes and remove lat and lon dims
        # 2) use this intermediate result to set up the prod generator used below
        # 3) add the lat and lon dims and sizes of target
        result_sizes = OrderedDict(da.sizes)
        for c in ['latitude', 'longitude']:
            _ = [result_sizes.pop(d, None) for d in da[c].dims]

        prod = list(product(*[range(d) for d in result_sizes.values()]))
        #print(result_sizes, prod)
        non_horiz_dims = result_sizes.keys()

        result_sizes.update(target['cells'].sizes)

        # loop over horizontal slices to avoid creation of temporary data array
        # with swapped axes for pkdt.resample_xyz
        result = np.ones(tuple(result_sizes.values()))
        for sel in prod:
            sel_dict = {dim: i for dim, i in zip(non_horiz_dims, sel)}
            print(f'Interpolate {sel_dict}')
            result[sel] = resample_func(source_geo, da[sel_dict].values, target_geo, *resample_args)
        
        coords = {'latitude': ('cells',target.coords['latitude'].values),
                  'longitude':('cells',target.coords['longitude'].values),
                  'cells':('cells',target.coords['cells'].values)
                  }
        if lev is not None:
            coords['height'] = lev

        target.values = result
        target = target.assign_coords(coords)
        return target

In [ ]:
lllat = xarray.DataArray(np.arange(-90,90, 2, dtype=np.float64), name='latitude',dims='cells')
lllon = xarray.zeros_like(lllat) + 10.
lllon.name = 'longitude'
lllat=lllat.assign_coords(dict(latitude=lllat, longitude=lllon))
lllon=lllon.assign_coords(dict(latitude=lllat, longitude=lllon))

In [ ]:
target = compute_xsection(full['T2M'],lllat,lllon)

In [ ]:
target

In [ ]:
target.plot(x='latitude');

# vertical crossection

## download some icond2: use today 00UTC init

In [ ]:
#!wget http://icon-downloads.mpimet.mpg.de/grids/public/edzw/icon_grid_0047_R19B07_L.nc

In [ ]:
#ifileP = 'icon-d2_germany_icosahedral_model-level_{date}_000_{lev}_qv.grib2.bz2'
#
#for lev in np.arange(65,45,-1):
#    subprocess.check_call(['wget', 
#     'http://opendata.dwd.de/weather/nwp/icon-d2/grib/00/qv/'+ ifileP.format(lev=lev,date=date)])
#    subprocess.check_call(['bzip2','-d',ifileP.format(lev=lev,date=date)])

In [ ]:
# ifileP = 'icon-d2_germany_icosahedral_time-invariant_{date}_000_{lev}_hhl.grib2.bz2'

# for lev in np.arange(65,45,-1):
#     subprocess.check_call(['wget', 
#      'http://opendata.dwd.de/weather/nwp/icon-d2/grib/00/hhl/'+ ifileP.format(lev=lev,date=date)])
#     subprocess.check_call(['bzip2','-d',ifileP.format(lev=lev,date=date)])

## read and quickview

In [ ]:
gridfID2 = 'icon_grid_0047_R19B07_L.nc'
gridD2 = xarray.open_dataset(gridfID2)

In [ ]:
qv = xarray.open_mfdataset(f'icon-d2_germany_icosahedral_model-level_{date}_000_??_qv.grib2', engine='cfgrib',
                          concat_dim='generalVerticalLayer', combine='nested', decode_timedelta=False)
qv = qv.rename(values='cell') # is called values....

In [ ]:
# ignoring that hhl is actually half levels here
hhl = xarray.open_mfdataset(f'./icon-d2_germany_icosahedral_time-invariant_{date}_000_??_hhl.grib2', engine='cfgrib',
                          concat_dim='generalVerticalLayer', combine='nested', decode_timedelta=False)
hhl = hhl.rename(values='cell')

In [ ]:
qv = qv.assign_coords(hhl=hhl['HHL'])

In [ ]:
fullID2 = xarray.Dataset(dict(QV=qv['QV']), coords=dict(latitude=np.rad2deg(gridD2.clat), longitude=np.rad2deg(gridD2.clon)))

In [ ]:
fullID2

In [ ]:
ax, tri, cax = plot_tri(qv['QV'].sel(generalVerticalLayer=65), gridD2,vmin=qv.QV.min().values, vmax=qv.QV.max().values);
#ax.set_extent([5,10,55,60])
plt.colorbar(cax, shrink=0.6, label=qv.QV.long_name)
plt.tight_layout()

## crossection

### do vertical interpolation first (1d problem)

In [ ]:
# target vertical heights to which we wish to interpolate
z = xarray.DataArray([20,100,250,500,750, 1000.], name='Z', dims=['Z'])

In [ ]:
@np.vectorize(signature='(m),(m),(p)->(p)',excluded={'xout'})
def interp1d(xin, yin, xout):
    return np.interp(xout, xin, yin,left=np.nan) # left: do not extrapolate below lowest model level

interpolat_array = interp1d(qv.hhl.values[::-1,:].T, fullID2.QV.values[::-1,:].T,z.values)
interpolatV = xarray.DataArray(interpolat_array, dims=('cell','Z'))
interpolatV = interpolatV.assign_coords(latitude=fullID2['latitude']) # automatically copies longitude as well

In [ ]:
interpolatV

In [ ]:
# check interpolation for a random column
plt.plot(qv.QV.values[::-1,50000],qv.hhl.values[::-1,50000], label='original profile')
plt.scatter(np.interp(z,qv.hhl.values[::-1,50000], qv.QV.values[::-1,50000]),z, label='single col interpolation') 
plt.scatter(interpolatV.isel(cell=50000),z, marker='x', label='batch computed interpolation')
plt.legend();

### do horizontal interpolation: example from Hamburg to Munich

In [ ]:
# choose transsect HH Munich
ham = [55, 10]
muc = [48, 12]

lllat = xarray.DataArray(np.linspace(ham[0], muc[0], 100, dtype=np.float64), name='latitude',dims='cells')
lllon = xarray.DataArray(np.linspace(ham[1], muc[1], 100, dtype=np.float64), name='longitude',dims='cells')
lllat=lllat.assign_coords(dict(latitude=lllat, longitude=lllon))
lllon=lllon.assign_coords(dict(latitude=lllat, longitude=lllon))

In [ ]:
tt = compute_xsection(interpolatV, lllat, lllon, method='gauss', sigma=20, cutoff=None, lev=z)

In [ ]:
tt

In [ ]:
tt.plot.contourf(x='latitude',y='height');

### for fun: interpolate horizontally without vertical interpolation to common height

In [ ]:
tt2 = compute_xsection(fullID2['QV'], lllat, lllon, method='nearest', sigma=20, cutoff=None, lev=fullID2.generalVerticalLayer)

In [ ]:
tt2.plot.contourf(x='latitude', y='generalVerticalLayer', yincrease=False);